# TwoPair Support Tests

Sanity checks for TwoPair claim ordering, parsing/rendering, call resolution, and dense BR smoke tests.


In [1]:
import os, sys
import time

def find_repo_root(start_dir: str) -> str:
    cur = os.path.abspath(start_dir)
    for _ in range(6):
        if os.path.isdir(os.path.join(cur, "liars_poker")) or os.path.exists(os.path.join(cur, "pyproject.toml")):
            return cur
        parent = os.path.dirname(cur)
        if parent == cur:
            break
        cur = parent
    return os.path.abspath(os.path.join(start_dir, "..", ".."))

NB_DIR = os.getcwd()
REPO_ROOT = find_repo_root(NB_DIR)
if REPO_ROOT not in sys.path:
    sys.path.insert(0, REPO_ROOT)

from liars_poker.core import GameSpec
from liars_poker.env import Rules, resolve_call_winner
from liars_poker.infoset import CALL
from liars_poker.policies.tabular_dense import DenseTabularPolicy
from liars_poker.algo.br_exact_dense_to_dense import best_response_dense

print("repo root:", REPO_ROOT)


repo root: c:\Users\adidh\Documents\liars_poker


In [2]:
# Ordering and parse/render checks
spec = GameSpec(ranks=4, suits=2, hand_size=2, claim_kinds=("RankHigh", "Pair", "TwoPair", "Trips"), suit_symmetry=True)
rules = Rules(spec)

claims = [rules.render_action(i) for i in range(len(rules.claims))]
print("Claims (ordered):")
print(claims)

idx_rh4 = rules.parse_action("RankHigh:4")
idx_pair1 = rules.parse_action("Pair:1")
idx_tp_12 = rules.parse_action("TwoPair:1,2")
idx_tp_13 = rules.parse_action("TwoPair:1,3")
idx_tp_23 = rules.parse_action("TwoPair:2,3")
idx_tp_14 = rules.parse_action("TwoPair:1,4")
idx_tp_24 = rules.parse_action("TwoPair:2,4")
idx_tp_34 = rules.parse_action("TwoPair:3,4")
idx_trips1 = rules.parse_action("Trips:1")

assert idx_rh4 < idx_pair1, "RankHigh block should precede Pair."
assert idx_pair1 < idx_tp_12, "Pair block should precede TwoPair."
assert idx_tp_12 < idx_tp_13 < idx_tp_23 < idx_tp_14 < idx_tp_24 < idx_tp_34, "TwoPair ordering mismatch."
assert idx_tp_34 < idx_trips1, "TwoPair block should precede Trips."

idx_roundtrip = rules.parse_action("TwoPair:4,2")
rendered = rules.render_action(idx_roundtrip)
print("Round-trip TwoPair:4,2 ->", rendered)
assert rendered == "TwoPair:4,2"


Claims (ordered):
['RankHigh:1', 'RankHigh:2', 'RankHigh:3', 'RankHigh:4', 'Pair:1', 'Pair:2', 'Pair:3', 'Pair:4', 'TwoPair:2,1', 'TwoPair:3,1', 'TwoPair:3,2', 'TwoPair:4,1', 'TwoPair:4,2', 'TwoPair:4,3', 'Trips:1', 'Trips:2', 'Trips:3', 'Trips:4']
Round-trip TwoPair:4,2 -> TwoPair:4,2


In [3]:
# TwoPair should be filtered when impossible (suits=1)
spec_bad = GameSpec(ranks=4, suits=1, hand_size=2, claim_kinds=("RankHigh", "TwoPair"), suit_symmetry=False)
rules_bad = Rules(spec_bad)
legal = [rules_bad.render_action(a) for a in rules_bad.legal_actions_from_last(None)]
print("Legal at root (suits=1):", legal)
assert all("TwoPair" not in s for s in legal)


Legal at root (suits=1): ['RankHigh:1', 'RankHigh:2', 'RankHigh:3', 'RankHigh:4']


In [4]:
# Call resolution: true vs false TwoPair
spec_call = GameSpec(ranks=4, suits=2, hand_size=2, claim_kinds=("RankHigh", "Pair", "TwoPair"), suit_symmetry=True)
rules_call = Rules(spec_call)
claim = rules_call.parse_action("TwoPair:2,3")
history = (claim, CALL)

# True: total has two 2s and two 3s
p1_hand = (2, 2)
p2_hand = (3, 3)
winner_true = resolve_call_winner(spec_call, history, p1_hand, p2_hand)
print("Winner (true claim):", winner_true)
assert winner_true == "P1"

# False: only one 3 total
p1_hand = (2, 2)
p2_hand = (2, 3)
winner_false = resolve_call_winner(spec_call, history, p1_hand, p2_hand)
print("Winner (false claim):", winner_false)
assert winner_false == "P2"


Winner (true claim): P1
Winner (false claim): P2


In [5]:
# Dense BR smoke test on a small TwoPair spec
spec_dense = GameSpec(ranks=3, suits=2, hand_size=2, claim_kinds=("RankHigh", "TwoPair"), suit_symmetry=True)
opp = DenseTabularPolicy(spec_dense)

print(f"k={opp.k}, H={1 << opp.k}, hands={len(opp.hands)}")
t0 = time.perf_counter()
br_pol, log = best_response_dense(spec_dense, opp, debug=False, store_state_values=False)
elapsed = time.perf_counter() - t0
print(f"BR solved in {elapsed:.3f}s")
assert br_pol.k == opp.k


k=6, H=64, hands=6
BR solved in 0.014s
